## Plot data

In [7]:
from pathlib import Path

import h5py
import numpy as np
import scipy as sp
import skimage as ski
import matplotlib.pyplot as plt
import os

from fusion import optimize
from utils import hyperspectral2RGB, hyperspectral2RGBvolume, bin_data
from utils import RESOURCES_PATH
from ipywidgets import interact

RESOURCES_PATH = Path("/Users/federicosimoni/Library/Mobile Documents/com~apple~CloudDocs/Università/Tesi/Code/CS-FLIM_lab/20240703/kidney")

DATA_PATH = RESOURCES_PATH / "520_kidney2_550_550_610_SPC_raw_proc_tlxy_fused.npz"

In [8]:
data = np.load(DATA_PATH)
x = data['x']
t = data['t']
lam = data['lam']

slices_rgb = hyperspectral2RGBvolume(lam, np.mean(x, axis=0))

means_spectrums = np.mean(x, axis=(0, 3, 4)).T
min_mean_spectrum = np.min(means_spectrums)
max_mean_spectrum = np.max(means_spectrums)

means_times = np.mean(x, axis=(1, 3, 4)).T
min_mean_times = np.min(means_times)
max_mean_times = np.max(means_times)

zxy = np.sum(x, axis=(0, 1))
zxy /= zxy.max()

def plot_across_z(index_z=0):
    _, ax = plt.subplots(2, 2, figsize=(8, 8))
    ax[0, 0].imshow(zxy[index_z], cmap="gray", vmin=0, vmax=1)
    ax[0, 0].set_title(f"Fused image, z={index_z}")

    ax[0, 1].imshow(slices_rgb[index_z])
    ax[0, 1].set_title(f"Colored with spectrum")

    ax[1, 0].plot(lam, means_spectrums[index_z])
    ax[1, 0].set_ylim(min_mean_spectrum, max_mean_spectrum)
    ax[1, 0].set_title(f"Fused Spectrum, z={index_z}")
    ax[1, 0].grid()

    ax[1, 1].plot(t, means_times[index_z])
    ax[1, 1].set_ylim(min_mean_times, max_mean_times)
    ax[1, 1].set_title(f"Fused Time, z={index_z}")
    ax[1, 1].grid()

    plt.tight_layout()
    plt.show()


interact(plot_across_z, index_z=(0, len(zxy) - 1, 1));

interactive(children=(IntSlider(value=0, description='index_z', max=14), Output()), _dom_classes=('widget-inte…

In [9]:
reconstructed_spectrums = np.mean(x, axis=(0, 2))
min_spectrums, max_spectrums = np.min(reconstructed_spectrums), np.max(reconstructed_spectrums)

reconstructed_times = np.mean(x, axis=(1, 2))
min_times, max_times = np.min(reconstructed_times), np.max(reconstructed_times)

def reconstructed_spectrum_time_in_a_point(i=0, j=0):
    _, ax = plt.subplots(1, 3, figsize=(12, 4))
    z_index = np.argmax(means_spectrums.mean(axis=1))
    ax[0].imshow(slices_rgb[z_index])
    ax[0].scatter([j], [i], c="b")
    ax[0].set_title(f"Colored with spectrum")
    
    ax[1].plot(lam, reconstructed_spectrums[:, i, j])
    ax[1].set_title(f"Spectrum in ({i},{j})")
    ax[1].set_ylim(min_spectrums, max_spectrums)
    ax[1].grid()
    
    ax[2].plot(t, reconstructed_times[:, i, j])
    ax[2].set_title(f"Time in ({i},{j})")
    ax[2].set_ylim(min_times, max_times)
    ax[2].grid()
    
    plt.tight_layout()
    plt.show()
    
interact(reconstructed_spectrum_time_in_a_point, i=(0, x.shape[-2], 1), j=(0, x.shape[-1], 1));

interactive(children=(IntSlider(value=0, description='i', max=256), IntSlider(value=0, description='j', max=25…